In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import glob
import os
import datetime
import seaborn as sns
sns.set(rc={'axes.facecolor':'gainsboro'})
import plotting_map_functions as plot_func
import functions_to_load_datasets as load_func
import functions_to_load_datasets as load_datasets

ERROR 1: PROJ: proj_create_from_database: Open of /home/atmo/bin/anaconda3/envs/lartelt_dev/share/proj failed


# First: other stuff, below: calc_ra+rh and so on

## ATTENTION: The calculated MEAN, STD, COUNT were calculated WRONG! DO NOT USE! For these calculations, the columns "MonthDate", "Month" and "Year" were INCLUDED, which is wrong!

### This error was corrected in the cells used for the arid/humid region

In [2]:
variable_list = ['gpp', 'npp', 'ra', 'rh', 'nee', 'nep', 'nbp']
variable_list_short = ['gpp', 'npp']
model_list = ['CABLE-POP', 'CLASSIC']
#means={}

for variable in variable_list_short:
    #means[variable+'_mean'] = []
    #means[variable+'_std'] = []
    #means[variable+'_count'] = []
    for i, model in enumerate(model_list):
        print('Calculate ensemble mean for '+variable+' of model '+model)
        if i==0:
            print('i=0')
            gdf_new = load_func.load_TRENDY_model_gdf(model=model, variable=variable, unit='TgC_per_region_per_month')
            gdf_new.rename(columns={variable+'tot_TgC_month': variable+'tot_TgC_month_'+model}, inplace=True)
        else:
            print('i = '+str(i))
            gdf = load_func.load_TRENDY_model_gdf(model=model, variable=variable, unit='TgC_per_region_per_month')
            gdf.rename(columns={variable+'tot_TgC_month': variable+'tot_TgC_month_'+model}, inplace=True)
            gdf_new = gdf_new.merge(gdf, on='MonthDate')
    gdf_old = gdf_new.copy()
    gdf_new['mean'] = gdf_new.mean(axis=1)
    gdf_new['std'] = gdf_old.std(ddof=0, axis=1)
    gdf_new['count'] = gdf_old.count(axis=1) - 1
    gdf_new['Month'] = gdf_new['MonthDate'].apply(lambda x: x.month)
    print(gdf_new)
    
    #gdf_final.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_'+variable+'_SAT_721.pkl')
    #print(gdf_final)



Calculate ensemble mean for gpp of model CABLE-POP
i=0
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
Calculate ensemble mean for gpp of model CLASSIC
i = 1
Done loading data for gdf1x1_CLASSIC_gpp_SAT_721.pkl
      MonthDate  gpptot_TgC_month_CABLE-POP  gpptot_TgC_month_CLASSIC  \
0    2008-01-15                 1394.303945               1196.294472   
1    2008-02-15                 1287.344708               1219.468642   
2    2008-03-15                 1284.566462               1302.589760   
3    2008-04-15                 1213.071124               1166.578587   
4    2008-05-15                 1188.908318               1018.189426   
..          ...                         ...                       ...   
139  2019-08-15                  979.565617                605.490349   
140  2019-09-15                  867.944724                547.304183   
141  2019-10-15                 1092.252074                730.948107   
142  2019-11-15                 1275.765210         

In [3]:
print(np.mean([741.059408,644.645823]))
print(np.std([741.059408,644.645823]))


692.8526155
48.206792500000006


In [4]:
df = pd.read_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_nbp_SAT_721.pkl')
df

start_month = 5
color_list = ['r', 'g', 'b', 'c', 'm', 'y', 'k', 'orange', 'purple', 'brown', 'pink', 'olive', 'cyan', 'lime', 'teal', 'darkred', 'darkgreen', 'darkblue', 'darkcyan', 'darkmagenta']
print('Start plotting...')
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw={'width_ratios':[8,3]})
fig.subplots_adjust(wspace=0)

first_month = start_month-2
for i, column in enumerate(df.columns):
    if column=='MonthDate' or column=='Month' or column=='count':
        print('column = '+column)
        continue
    else:
        ax1.plot(df.MonthDate, df[column], color = color_list[i], linestyle='solid', label=column, linewidth=1.25)
        print(i)
ax1.legend(loc='upper left',fontsize=8)
plt.savefig('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/timeseries_all_models_try', dpi=300, bbox_inches='tight')
        

Start plotting...
column = MonthDate
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
column = count
column = Month


In [5]:
dfy=load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable='nbp', unit='mean_of_variable', start_year=2009)

for i, column in enumerate(dfy.columns):
        if column=='MonthDate' or column=='Year' or column=='Month' or column=='count':
            print('column = '+column)
        else:
            print(column[17:])
            print(column[:3])

Done loading data for gdf1x1_CABLE-POP_nbp_SAT_721.pkl
column = MonthDate
CLASSIC
nbp
IBIS
nbp
ISAM
nbp
JSBACH
nbp
LPX-Bern
nbp
OCN
nbp
ORCHIDEE
nbp
ORCHIDEE-CNP
nbp
ORCHIDEEv3
nbp
CLM5.0
nbp
ISBA-CTRIP
nbp
JULES-ES-1p0
nbp
LPJ
nbp
SDGVM
nbp
VISIT
nbp
YIBs
nbp

mea

std
column = count
column = Month
column = Year


In [6]:
Var='fFire'
df_TM5_IS_ACOS = load_datasets.return_dataframe_from_name_TM5_gridded_3x2_CT_Mask('IS_ACOS', region='SAT', unit='per_subregion')
df_TM5_IS_RT = load_datasets.return_dataframe_from_name_TM5_gridded_3x2_CT_Mask('IS_RT', region='SAT', unit='per_subregion')
df_MIP_ens = load_datasets.return_dataframe_from_name_MIP_gridded_CT_Mask(model='MIP_ens', assimilated='IS_OCO2', end_year=2019, region='SAT', unit='per_subregion')

df_TRENDY = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=Var, unit='mean_of_variable', start_year=2009)

print(df_TRENDY)


loading dataset TM5-4DVar_IS_ACOS_flux_gridded_3x2_per_subregion_region_SAT_CT_Mask from path /mnt/data/users/lartelt/MA/Datasets_cut_with_CT_Mask/TM5-4DVar_flux_gridded_3x2_from_coarsen/TM5-4DVar_IS_ACOS_ass_flux_gridded_3x2_per_subregion_SAT_CT_Mask.pkl
Done loading dataset
loading dataset TM5-4DVar_IS_RT_flux_gridded_3x2_per_subregion_region_SAT_CT_Mask from path /mnt/data/users/lartelt/MA/Datasets_cut_with_CT_Mask/TM5-4DVar_flux_gridded_3x2_from_coarsen/TM5-4DVar_IS_RT_ass_flux_gridded_3x2_per_subregion_SAT_CT_Mask.pkl
Done loading dataset
loading dataset MIP_ens IS_OCO2_assimilated in SAT region from path /mnt/data/users/lartelt/MA/Datasets_cut_with_CT_Mask/MIP_ens_flux/MIP_ens_IS_OCO2_ass_flux_gridded_1x1_per_subregion_SAT_CT_Mask.pkl
Done loading dataset
Done loading data for gdf1x1_CABLE-POP_fFire_SAT_721.pkl
      MonthDate  fFiretot_TgC_month_CLASSIC  fFiretot_TgC_month_JSBACH  \
12   2009-01-15                   27.931130                  21.568733   
13   2009-02-15        

In [7]:
df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable='ra', unit='mean_of_variable', start_year=2009)
df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable='rh', unit='mean_of_variable', start_year=2009)

df = pd.merge(df_TRENDY_var1, df_TRENDY_var2, how='outer', on='MonthDate')

print(df['ratot_TgC_month_CABLE-POP'])
print(df['rhtot_TgC_month_CABLE-POP'])

print(df['ratot_TgC_month_CABLE-POP'] + df['rhtot_TgC_month_CABLE-POP'])

Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
0      792.815851
1      716.009956
2      755.186213
3      677.783126
4      647.376490
          ...    
127    564.269993
128    492.901052
129    551.786234
130    626.385850
131    729.929893
Name: ratot_TgC_month_CABLE-POP, Length: 132, dtype: float64
0      689.984609
1      603.882357
2      609.066507
3      509.907596
4      455.695140
          ...    
127    429.317520
128    470.589181
129    584.043711
130    618.100519
131    692.500610
Name: rhtot_TgC_month_CABLE-POP, Length: 132, dtype: float64
0      1482.800460
1      1319.892314
2      1364.252719
3      1187.690723
4      1103.071630
          ...     
127     993.587513
128     963.490233
129    1135.829945
130    1244.486370
131    1422.430502
Length: 132, dtype: float64


In [8]:
df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable='ra', unit='mean_of_variable', start_year=2009)
df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable='rh', unit='mean_of_variable', start_year=2009)
models_to_plot = ['CABLE-POP', 'CLASSIC']
var1 = 'ra'
var2 = 'rh'

df_TRENDY = pd.merge(df_TRENDY_var1, df_TRENDY_var2, on=['MonthDate'], how='outer')
df_TRENDY_sum = df_TRENDY[['MonthDate']]
df_TRENDY_sum['Month'] = df_TRENDY_sum.apply(lambda x: x['MonthDate'].month, axis=1)
for model in models_to_plot:
    df_TRENDY_sum[var1+'tot+'+var2+'tot_TgC_month_'+model] = df_TRENDY[var1+'tot_TgC_month_'+model] + df_TRENDY[var2+'tot_TgC_month_'+model]
print(df_TRENDY_sum)

Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
      MonthDate  Month  ratot+rhtot_TgC_month_CABLE-POP  \
0    2009-01-15      1                      1482.800460   
1    2009-02-15      2                      1319.892314   
2    2009-03-15      3                      1364.252719   
3    2009-04-15      4                      1187.690723   
4    2009-05-15      5                      1103.071630   
..          ...    ...                              ...   
127  2019-08-15      8                       993.587513   
128  2019-09-15      9                       963.490233   
129  2019-10-15     10                      1135.829945   
130  2019-11-15     11                      1244.486370   
131  2019-12-15     12                      1422.430502   

     ratot+rhtot_TgC_month_CLASSIC  
0                      1137.915605  
1                      1063.750608  
2                      1133.064972  
3                       981.114173 

# For TRENDY region SAT

## Calculate ra+rh

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import glob
import os
import datetime
import seaborn as sns
sns.set(rc={'axes.facecolor':'gainsboro'})
import plotting_map_functions as plot_func
import functions_to_load_datasets as load_func
import functions_to_load_datasets as load_datasets

var1='ra'
var2='rh'
df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var1, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
#print(df_TRENDY_var1.head())
df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var2, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var2 = df_TRENDY_var2.columns.drop(['MonthDate', 'Month', 'Year'])
#print(df_TRENDY_var2.head())
#df_TRENDY = pd.merge(df_TRENDY_var1, df_TRENDY_var2, on=['MonthDate', 'Month', 'Year'], how='outer')
df_TRENDY_sum = df_TRENDY_var1[['MonthDate', 'Month', 'Year']]

model_list = []
for column in columns_var1:
    col = 'rh'+column[2:]
    if col in columns_var2:
        model_list.append(column[16:])
print(model_list)

for model in model_list:
    df_TRENDY_sum[var1+'tot+'+var2+'tot_TgC_month_'+model] = df_TRENDY_var1[var1+'tot_TgC_month_'+model] + df_TRENDY_var2[var2+'tot_TgC_month_'+model]

df_TRENDY_sum_old = df_TRENDY_sum.copy()
df_TRENDY_sum['mean'] = df_TRENDY_sum.mean(axis=1)
df_TRENDY_sum['std'] = df_TRENDY_sum_old.std(axis=1)
df_TRENDY_sum['count'] = df_TRENDY_sum_old.count(axis=1)
#print(df_TRENDY_sum.head())
#save df_TRENDY_sum to pickle

df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_ra+rh_SAT_721.pkl')




ERROR 1: PROJ: proj_create_from_database: Open of /home/atmo/bin/anaconda3/envs/lartelt_dev/share/proj failed


Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
['CABLE-POP', 'CLASSIC', 'DLEM', 'IBIS', 'ISAM', 'JSBACH', 'LPX-Bern', 'OCN', 'ORCHIDEE', 'ORCHIDEE-CNP', 'ORCHIDEEv3', 'CLM5.0', 'ISBA-CTRIP', 'LPJ', 'SDGVM', 'VISIT', 'YIBs']


## Calculate ra+rh-gpp

#### Previously ra+rh+gpp but thats not good for comparison

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import seaborn as sns
sns.set(rc={'axes.facecolor':'gainsboro'})
import plotting_map_functions as plot_func
import functions_to_load_datasets as load_func
import functions_to_load_datasets as load_datasets

var1='ra+rh'
var1_column_name='ratot+rh'
var2='gpp'
var2_column_name='gpp'

df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var1, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
print(df_TRENDY_var1.head())
df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var2, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var2 = df_TRENDY_var2.columns.drop(['MonthDate', 'Month', 'Year'])
print(df_TRENDY_var2.head())
#df_TRENDY = pd.merge(df_TRENDY_var1, df_TRENDY_var2, on=['MonthDate', 'Month', 'Year'], how='outer')

df_TRENDY_sum = df_TRENDY_var1[['MonthDate', 'Month', 'Year']]

model_list = []
for column in columns_var1:
    col = 'gpp'+column[8:]
    #print(col)
    if col in columns_var2:
        model_list.append(column[22:])
#print(len(columns_var1))
#print(len(columns_var2))
#print(len(model_list))

for model in model_list:
    df_TRENDY_sum[var1_column_name+'tot-'+var2_column_name+'tot_TgC_month_'+model] = df_TRENDY_var1[var1_column_name+'tot_TgC_month_'+model] - df_TRENDY_var2[var2_column_name+'tot_TgC_month_'+model]

df_TRENDY_sum_old = df_TRENDY_sum.copy()
df_TRENDY_sum['mean'] = df_TRENDY_sum.mean(axis=1)
df_TRENDY_sum['std'] = df_TRENDY_sum_old.std(axis=1)
df_TRENDY_sum['count'] = df_TRENDY_sum_old.count(axis=1)
#print(df_TRENDY_sum.head())
#save df_TRENDY_sum to pickle
df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_ra+rh-gpp_SAT_721.pkl')
print(df_TRENDY_sum.head())


ERROR 1: PROJ: proj_create_from_database: Open of /home/atmo/bin/anaconda3/envs/lartelt_dev/share/proj failed


Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
    MonthDate  Month  Year  ratot+rhtot_TgC_month_CABLE-POP  \
0  2008-01-15      1  2008                      1405.098850   
1  2008-02-15      2  2008                      1300.901549   
2  2008-03-15      3  2008                      1289.534137   
3  2008-04-15      4  2008                      1143.463479   
4  2008-05-15      5  2008                      1039.899808   

   ratot+rhtot_TgC_month_CLASSIC  ratot+rhtot_TgC_month_DLEM  \
0                    1155.139581                 1331.189813   
1                    1091.417346                 1296.102627   
2                    1104.859846                 1348.911444   
3                     971.923217                 1207.794703   
4                     865.351742                 1088.277478   

   ratot+rhtot_TgC_month_IBIS  ratot+rhtot_TgC_month_ISAM  \
0                 1355.692083                 1291.160874   
1                 1369.263039                 1203.704933

## Calculate nbp+gpp-(ra+rh) = Fire


nbp = fire + ra + rh - gpp  or  -nbp = fire + ra + rh - gpp

nbp - ra - rh + gpp = fire  or  -nbp - ra - rh + gpp = fire

In [13]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import seaborn as sns
sns.set(rc={'axes.facecolor':'gainsboro'})
import plotting_map_functions as plot_func
import functions_to_load_datasets as load_func
import functions_to_load_datasets as load_datasets

var1='ra+rh'
var1_column_name='ratot+rh'
var2='gpp'
var2_column_name='gpp'
var3='nbp'
var3_column_name='nbp'

df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var1, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
print(df_TRENDY_var1.head())
df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var2, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var2 = df_TRENDY_var2.columns.drop(['MonthDate', 'Month', 'Year'])
print(df_TRENDY_var2.head())
df_TRENDY_var3 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var3, unit='mean_of_variable').drop(columns=['mean', 'std', 'count'])
columns_var3 = df_TRENDY_var3.columns.drop(['MonthDate', 'Month', 'Year'])
print(df_TRENDY_var3.head())

df_TRENDY_sum = df_TRENDY_var1[['MonthDate', 'Month', 'Year']]

model_list = []
for column in columns_var1:
    #print(column) # column = 'ratot+rhtot_TgC_month_CABLE-POP'
    col2 = 'gpp'+column[8:] # 8 due to 'ratot+rh': gpp+column[8:] = gpptot_TgC_month_CABLE-POP
    col3 = 'nbp'+column[8:] # 8 due to 'ratot+rh': nbp+column[8:] = nbptot_TgC_month_CABLE-POP
    #print(col2)
    #print(col3)
    if col2 in columns_var2 and col3 in columns_var3:
        model_list.append(column[22:]) # column[22:] = CABLE-POP
#print(len(columns_var1))
#print(len(columns_var2))
#print(len(model_list))

for model in model_list:
    # nbp-(ra+rh)+gpp
    df_TRENDY_sum[var3_column_name+'tot-('+var1_column_name+'tot)+'+var2_column_name+'tot_TgC_month_'+model] = (
        df_TRENDY_var3[var3_column_name+'tot_TgC_month_'+model] -
        df_TRENDY_var1[var1_column_name+'tot_TgC_month_'+model] +
        df_TRENDY_var2[var2_column_name+'tot_TgC_month_'+model])

df_TRENDY_sum_old = df_TRENDY_sum.copy()
df_TRENDY_sum['mean'] = df_TRENDY_sum.mean(axis=1)
df_TRENDY_sum['std'] = df_TRENDY_sum_old.std(axis=1)
df_TRENDY_sum['count'] = df_TRENDY_sum_old.count(axis=1)
#print(df_TRENDY_sum.head())
#save df_TRENDY_sum to pickle
df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_nbp-(ra+rh)+gpp_SAT_721.pkl')
print(df_TRENDY_sum.head())


Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
    MonthDate  Month  Year  ratot+rhtot_TgC_month_CABLE-POP  \
0  2008-01-15      1  2008                      1405.098850   
1  2008-02-15      2  2008                      1300.901549   
2  2008-03-15      3  2008                      1289.534137   
3  2008-04-15      4  2008                      1143.463479   
4  2008-05-15      5  2008                      1039.899808   

   ratot+rhtot_TgC_month_CLASSIC  ratot+rhtot_TgC_month_DLEM  \
0                    1155.139581                 1331.189813   
1                    1091.417346                 1296.102627   
2                    1104.859846                 1348.911444   
3                     971.923217                 1207.794703   
4                     865.351742                 1088.277478   

   ratot+rhtot_TgC_month_IBIS  ratot+rhtot_TgC_month_ISAM  \
0                 1355.692083                 1291.160874   
1                 1369.263039                 1203.704933

# Calculate ra+rh-gpp+GFED_fire

In [12]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
sns.set(rc={'axes.facecolor':'gainsboro'})
import plotting_map_functions as plot_func
import functions_to_load_datasets as load_func
import functions_to_load_datasets as load_datasets

var1='ra+rh-gpp'
var1_column_name='ratot+rhtot-gpp'

df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(model='CABLE-POP', variable=var1, unit='mean_of_variable', start_year=2009, end_year=2019).drop(columns=['mean', 'std', 'count'])
columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
#print(df_TRENDY_var1)

GFED = load_datasets.load_GFED_model_gdf(region='SAT', unit='per_subregion', start_year=2009, end_year=2019).drop(columns=['Month', 'Year'])
#print(GFED)

df_TRENDY_var1_merged = df_TRENDY_var1.merge(GFED, on='MonthDate', how='outer')
df_TRENDY_sum = df_TRENDY_var1_merged[['MonthDate', 'Month', 'Year']]

model_list = []
for column in columns_var1:
    #print(column) # column = 'ratot+rhtot_TgC_month_CABLE-POP'
    model_list.append(column[29:]) # column[22:] = CABLE-POP

for model in model_list:
    # ra+rh-gpp+GFED
    df_TRENDY_sum[var1_column_name+'tot+GFED_TgC_month_'+model] = (df_TRENDY_var1_merged[var1_column_name+'tot_TgC_month_'+model] + 
                                                                   df_TRENDY_var1_merged['total_emission'])

df_for_mean = df_TRENDY_sum.drop(columns=['MonthDate', 'Month', 'Year'])
df_TRENDY_sum['mean'] = df_for_mean.mean(axis=1)
df_TRENDY_sum['std'] = df_for_mean.std(axis=1)
df_TRENDY_sum['count'] = df_for_mean.count(axis=1)
print(df_TRENDY_sum)

df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_ra+rh-gpp+GFED_SAT_721.pkl')


Done loading data for gdf1x1_CABLE-POP_ra+rh-gpp_SAT_721.pkl
Done loading data from /mnt/data/users/lartelt/MA/GFED4/GFED_cut_with_CT_Mask/ GFED_CT_Mask_SAT_TgC_per_month_per_subregion.pkl
      MonthDate  Month  Year  ratot+rhtot-gpptot+GFED_TgC_month_CABLE-POP  \
0    2009-01-15      1  2009                                   -49.086330   
1    2009-02-15      2  2009                                   -26.908376   
2    2009-03-15      3  2009                                   -10.502002   
3    2009-04-15      4  2009                                   -74.945237   
4    2009-05-15      5  2009                                  -107.736973   
..          ...    ...   ...                                          ...   
127  2019-08-15      8  2019                                    50.204443   
128  2019-09-15      9  2019                                   143.818003   
129  2019-10-15     10  2019                                    60.033479   
130  2019-11-15     11  2019             

## TM5/IS+ACOS nbp - GFED

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
sns.set(rc={'axes.facecolor':'gainsboro'})
import plotting_map_functions as plot_func
import functions_to_load_datasets as load_func
import functions_to_load_datasets as load_datasets

df_TM5_IS_ACOS = load_datasets.return_dataframe_from_name_TM5_gridded_3x2_CT_Mask('IS_ACOS', region='SAT', unit='per_subregion')
#print(df_TM5_IS_ACOS)
df_GFED = load_datasets.load_GFED_model_gdf(region='SAT', unit='per_subregion', start_year=2009, end_year=2019).drop(columns=['Month', 'Year'])
#print(df_GFED[:len(df_GFED['total_emission'])-6])

df_TM5_IS_ACOS_merged = df_TM5_IS_ACOS.merge(df_GFED[:len(df_GFED['total_emission'])-6], on='MonthDate', how='outer')

df_TM5_IS_ACOS_merged['CO2_NEE_fire_MINUS_GFED_flux_monthly_TgC_per_subregion'] = (df_TM5_IS_ACOS_merged['CO2_NEE_fire_flux_monthly_TgC_per_subregion'] - 
                                                                                   df_TM5_IS_ACOS_merged['total_emission'])
#print(df_TM5_IS_ACOS_merged)

df_TM5_IS_ACOS_merged.to_pickle('/mnt/data/users/lartelt/MA/Datasets_cut_with_CT_Mask/TM5-4DVar_flux_gridded_3x2_from_coarsen/TM5-4DVar_IS_ACOS_MINUS_GFED_ass_flux_gridded_3x2_per_subregion_SAT_CT_Mask.pkl')


ERROR 1: PROJ: proj_create_from_database: Open of /home/atmo/bin/anaconda3/envs/lartelt_dev/share/proj failed


loading dataset TM5-4DVar_IS_ACOS_flux_gridded_3x2_per_subregion_region_SAT_CT_Mask from path /mnt/data/users/lartelt/MA/Datasets_cut_with_CT_Mask/TM5-4DVar_flux_gridded_3x2_from_coarsen/TM5-4DVar_IS_ACOS_ass_flux_gridded_3x2_per_subregion_SAT_CT_Mask.pkl
Done loading dataset
Done loading data from /mnt/data/users/lartelt/MA/GFED4/GFED_cut_with_CT_Mask/ GFED_CT_Mask_SAT_TgC_per_month_per_subregion.pkl
      MonthDate  total_emission
0    2009-01-15        2.337492
1    2009-02-15        0.787255
2    2009-03-15        1.835584
3    2009-04-15        3.429533
4    2009-05-15        1.966660
..          ...             ...
121  2019-02-15        2.391590
122  2019-03-15        1.405564
123  2019-04-15        0.851989
124  2019-05-15        1.275145
125  2019-06-15        3.065996

[126 rows x 2 columns]


# For TRENDY arid/humid region

## ra+rh

In [1]:
import numpy as np
import pandas as pd
import geopandas
import warnings
warnings.filterwarnings('ignore')
import functions_to_load_datasets as load_datasets

var1='ra'
var2='rh'
arid_type_list = ['looselimit_arid', 'looselimit_humid', 'strictlimit_arid', 'strictlimit_humid']
#arid_type_list = ['looselimit_arid']
for aridity in arid_type_list:
    df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(variable=var1, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    #print(df_TRENDY_var1.head())
    columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
    df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(variable=var2, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    #print(df_TRENDY_var2.head())
    columns_var2 = df_TRENDY_var2.columns.drop(['MonthDate', 'Month', 'Year'])

    df_TRENDY_sum = df_TRENDY_var1[['MonthDate', 'Month', 'Year']]

    model_list = []
    for column in columns_var1:
        col = 'rh'+column[2:]
        if col in columns_var2:
            model_list.append(column[16:])
    print(model_list)

    for model in model_list:
        df_TRENDY_sum[var1+'tot+'+var2+'tot_TgC_month_'+model] = df_TRENDY_var1[var1+'tot_TgC_month_'+model] + df_TRENDY_var2[var2+'tot_TgC_month_'+model]

    df_TRENDY_for_mean = df_TRENDY_sum.copy().drop(columns=['MonthDate', 'Month', 'Year'])
    df_TRENDY_sum['mean'] = df_TRENDY_for_mean.mean(axis=1)
    df_TRENDY_sum['std'] = df_TRENDY_for_mean.std(axis=1)
    df_TRENDY_sum['count'] = df_TRENDY_for_mean.count(axis=1)
    #print(df_TRENDY_sum.head())
    
    #save df_TRENDY_sum to pickle
    df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_ra+rh_SAT_'+aridity+'.pkl')


ERROR 1: PROJ: proj_create_from_database: Open of /home/atmo/bin/anaconda3/envs/lartelt_dev/share/proj failed


Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS', 'SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS', 'SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS', 'SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS'

## ra+rh-gpp

In [2]:
import numpy as np
import pandas as pd
import geopandas
import functions_to_load_datasets as load_datasets
import warnings
warnings.filterwarnings('ignore')

var1='ra+rh'
var1_column_name='ratot+rh'
var2='gpp'
var2_column_name='gpp'

arid_type_list = ['looselimit_arid', 'looselimit_humid', 'strictlimit_arid', 'strictlimit_humid']
#arid_type_list = ['looselimit_arid']
for aridity in arid_type_list:
    df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(variable=var1, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
    #print(df_TRENDY_var1)
    df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(variable=var2, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    columns_var2 = df_TRENDY_var2.columns.drop(['MonthDate', 'Month', 'Year'])
    #print(df_TRENDY_var2)

    df_TRENDY_sum = df_TRENDY_var1[['MonthDate', 'Month', 'Year']]

    model_list = []
    for column in columns_var1:
        col = 'gpp'+column[8:]
        #print(col)
        if col in columns_var2:
            model_list.append(column[22:])
    print(model_list)
    
    for model in model_list:
        df_TRENDY_sum[var1_column_name+'tot-'+var2_column_name+'tot_TgC_month_'+model] = df_TRENDY_var1[var1_column_name+'tot_TgC_month_'+model] - df_TRENDY_var2[var2_column_name+'tot_TgC_month_'+model]

    df_TRENDY_for_mean = df_TRENDY_sum.copy().drop(columns=['MonthDate', 'Month', 'Year'])
    df_TRENDY_sum['mean'] = df_TRENDY_for_mean.mean(axis=1)
    df_TRENDY_sum['std'] = df_TRENDY_for_mean.std(axis=1)
    df_TRENDY_sum['count'] = df_TRENDY_for_mean.count(axis=1)
    #print(df_TRENDY_sum)
    
    df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_ra+rh-gpp_SAT_'+aridity+'.pkl')
    
    
    

Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS', 'SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS', 'SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS', 'SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 

## nbp-(ra+rh)+gpp = fFire

In [3]:
import numpy as np
import pandas as pd
import geopandas
import warnings
warnings.filterwarnings('ignore')
import functions_to_load_datasets as load_datasets

var1='ra+rh'
var1_column_name='ratot+rh'
var2='gpp'
var2_column_name='gpp'
var3='nbp'
var3_column_name='nbp'

arid_type_list = ['looselimit_arid', 'looselimit_humid', 'strictlimit_arid', 'strictlimit_humid']
#arid_type_list = ['looselimit_arid']
for aridity in arid_type_list:
    df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(variable=var1, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
    #print(df_TRENDY_var1.head())
    df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(variable=var2, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    columns_var2 = df_TRENDY_var2.columns.drop(['MonthDate', 'Month', 'Year'])
    #print(df_TRENDY_var2.head())
    df_TRENDY_var3 = load_datasets.load_TRENDY_model_gdf(variable=var3, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    columns_var3 = df_TRENDY_var3.columns.drop(['MonthDate', 'Month', 'Year'])
    #print(df_TRENDY_var3.head())

    df_TRENDY_sum = df_TRENDY_var1[['MonthDate', 'Month', 'Year']]

    model_list = []
    for column in columns_var1:
        col2 = 'gpp'+column[8:] # 8 due to 'ratot+rh': gpp+column[8:] = gpptot_TgC_month_CABLE-POP
        col3 = 'nbp'+column[8:] # 8 due to 'ratot+rh': nbp+column[8:] = nbptot_TgC_month_CABLE-POP
        if col2 in columns_var2 and col3 in columns_var3:
            model_list.append(column[22:]) # column[22:] = CABLE-POP

    for model in model_list:
        # nbp-(ra+rh)+gpp
        df_TRENDY_sum[var3_column_name+'tot-('+var1_column_name+'tot)+'+var2_column_name+'tot_TgC_month_'+model] = (
            df_TRENDY_var3[var3_column_name+'tot_TgC_month_'+model] -
            df_TRENDY_var1[var1_column_name+'tot_TgC_month_'+model] +
            df_TRENDY_var2[var2_column_name+'tot_TgC_month_'+model])

    df_TRENDY_for_mean = df_TRENDY_sum.copy().drop(columns=['MonthDate', 'Month', 'Year'])
    df_TRENDY_sum['mean'] = df_TRENDY_for_mean.mean(axis=1)
    df_TRENDY_sum['std'] = df_TRENDY_for_mean.std(axis=1)
    df_TRENDY_sum['count'] = df_TRENDY_for_mean.count(axis=1)
    #print(df_TRENDY_sum.head())
    
    df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_nbp-(ra+rh)+gpp_SAT_'+aridity+'.pkl')
    

Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_nbp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_nbp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_nbp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_nbp_SAT_721.pkl


# For TRENDY arid subregions

## ra+rh

In [1]:
import numpy as np
import pandas as pd
import geopandas
import warnings
warnings.filterwarnings('ignore')
import functions_to_load_datasets as load_datasets

var1='ra'
var2='rh'
arid_type_list = ['looselimit_arid_west', 'looselimit_arid_east', 'strictlimit_arid_west', 'strictlimit_arid_east']
#arid_type_list = ['looselimit_arid_west']
for aridity in arid_type_list:
    df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(variable=var1, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    #print(df_TRENDY_var1.head())
    columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
    df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(variable=var2, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    #print(df_TRENDY_var2.head())
    columns_var2 = df_TRENDY_var2.columns.drop(['MonthDate', 'Month', 'Year'])

    df_TRENDY_sum = df_TRENDY_var1[['MonthDate', 'Month', 'Year']]

    model_list = []
    for column in columns_var1:
        col = 'rh'+column[2:]
        if col in columns_var2:
            model_list.append(column[16:])
    print(model_list)

    for model in model_list:
        df_TRENDY_sum[var1+'tot+'+var2+'tot_TgC_month_'+model] = df_TRENDY_var1[var1+'tot_TgC_month_'+model] + df_TRENDY_var2[var2+'tot_TgC_month_'+model]

    df_TRENDY_for_mean = df_TRENDY_sum.copy().drop(columns=['MonthDate', 'Month', 'Year'])
    df_TRENDY_sum['mean'] = df_TRENDY_for_mean.mean(axis=1)
    df_TRENDY_sum['std'] = df_TRENDY_for_mean.std(axis=1)
    df_TRENDY_sum['count'] = df_TRENDY_for_mean.count(axis=1)
    #print(df_TRENDY_sum.head())
    
    #save df_TRENDY_sum to pickle
    df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_ra+rh_SAT_'+aridity+'.pkl')

ERROR 1: PROJ: proj_create_from_database: Open of /home/atmo/bin/anaconda3/envs/lartelt_dev/share/proj failed


Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS', 'SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS', 'SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS', 'SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
Done loading data for gdf1x1_CABLE-POP_ra_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_rh_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS'

## ra+rh-gpp

In [1]:
import numpy as np
import pandas as pd
import geopandas
import functions_to_load_datasets as load_datasets
import warnings
warnings.filterwarnings('ignore')

var1='ra+rh'
var1_column_name='ratot+rh'
var2='gpp'
var2_column_name='gpp'

arid_type_list = ['looselimit_arid_west', 'looselimit_arid_east', 'strictlimit_arid_west', 'strictlimit_arid_east']
#arid_type_list = ['looselimit_arid_west']
for aridity in arid_type_list:
    df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(variable=var1, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
    #print(df_TRENDY_var1)
    df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(variable=var2, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    columns_var2 = df_TRENDY_var2.columns.drop(['MonthDate', 'Month', 'Year'])
    #print(df_TRENDY_var2)

    df_TRENDY_sum = df_TRENDY_var1[['MonthDate', 'Month', 'Year']]

    model_list = []
    for column in columns_var1:
        col = 'gpp'+column[8:]
        #print(col)
        if col in columns_var2:
            model_list.append(column[22:])
    print(model_list)
    
    for model in model_list:
        df_TRENDY_sum[var1_column_name+'tot-'+var2_column_name+'tot_TgC_month_'+model] = df_TRENDY_var1[var1_column_name+'tot_TgC_month_'+model] - df_TRENDY_var2[var2_column_name+'tot_TgC_month_'+model]

    df_TRENDY_for_mean = df_TRENDY_sum.copy().drop(columns=['MonthDate', 'Month', 'Year'])
    df_TRENDY_sum['mean'] = df_TRENDY_for_mean.mean(axis=1)
    df_TRENDY_sum['std'] = df_TRENDY_for_mean.std(axis=1)
    df_TRENDY_sum['count'] = df_TRENDY_for_mean.count(axis=1)
    #print(df_TRENDY_sum)
    
    df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_ra+rh-gpp_SAT_'+aridity+'.pkl')
    

ERROR 1: PROJ: proj_create_from_database: Open of /home/atmo/bin/anaconda3/envs/lartelt_dev/share/proj failed


Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS', 'SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS', 'SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 'JSBACH', 'IBIS', 'SDGVM', 'VISIT', 'LPX-Bern', 'CLM5.0', 'ORCHIDEEv3']
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
['CLASSIC', 'OCN', 'ISBA-CTRIP', 'ISAM', 'YIBs', 'ORCHIDEE', 'ORCHIDEE-CNP', 'LPJ', 

## nbp-(ra+rh)+gpp = fFire

In [1]:
import numpy as np
import pandas as pd
import geopandas
import warnings
warnings.filterwarnings('ignore')
import functions_to_load_datasets as load_datasets

var1='ra+rh'
var1_column_name='ratot+rh'
var2='gpp'
var2_column_name='gpp'
var3='nbp'
var3_column_name='nbp'

arid_type_list = ['looselimit_arid_west', 'looselimit_arid_east', 'strictlimit_arid_west', 'strictlimit_arid_east']
#arid_type_list = ['looselimit_arid_west']
for aridity in arid_type_list:
    df_TRENDY_var1 = load_datasets.load_TRENDY_model_gdf(variable=var1, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    columns_var1 = df_TRENDY_var1.columns.drop(['MonthDate', 'Month', 'Year'])
    #print(df_TRENDY_var1.head())
    df_TRENDY_var2 = load_datasets.load_TRENDY_model_gdf(variable=var2, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    columns_var2 = df_TRENDY_var2.columns.drop(['MonthDate', 'Month', 'Year'])
    #print(df_TRENDY_var2.head())
    df_TRENDY_var3 = load_datasets.load_TRENDY_model_gdf(variable=var3, unit='mean_of_variable', arid_type=aridity).drop(columns=['mean', 'std', 'count'])
    columns_var3 = df_TRENDY_var3.columns.drop(['MonthDate', 'Month', 'Year'])
    #print(df_TRENDY_var3.head())

    df_TRENDY_sum = df_TRENDY_var1[['MonthDate', 'Month', 'Year']]

    model_list = []
    for column in columns_var1:
        col2 = 'gpp'+column[8:] # 8 due to 'ratot+rh': gpp+column[8:] = gpptot_TgC_month_CABLE-POP
        col3 = 'nbp'+column[8:] # 8 due to 'ratot+rh': nbp+column[8:] = nbptot_TgC_month_CABLE-POP
        if col2 in columns_var2 and col3 in columns_var3:
            model_list.append(column[22:]) # column[22:] = CABLE-POP

    for model in model_list:
        # nbp-(ra+rh)+gpp
        df_TRENDY_sum[var3_column_name+'tot-('+var1_column_name+'tot)+'+var2_column_name+'tot_TgC_month_'+model] = (
            df_TRENDY_var3[var3_column_name+'tot_TgC_month_'+model] -
            df_TRENDY_var1[var1_column_name+'tot_TgC_month_'+model] +
            df_TRENDY_var2[var2_column_name+'tot_TgC_month_'+model])

    df_TRENDY_for_mean = df_TRENDY_sum.copy().drop(columns=['MonthDate', 'Month', 'Year'])
    df_TRENDY_sum['mean'] = df_TRENDY_for_mean.mean(axis=1)
    df_TRENDY_sum['std'] = df_TRENDY_for_mean.std(axis=1)
    df_TRENDY_sum['count'] = df_TRENDY_for_mean.count(axis=1)
    #print(df_TRENDY_sum.head())
    
    df_TRENDY_sum.to_pickle('/mnt/data/users/lartelt/MA/TRENDY/mean_of_all_models/Ensemble_mean_for_nbp-(ra+rh)+gpp_SAT_'+aridity+'.pkl')

ERROR 1: PROJ: proj_create_from_database: Open of /home/atmo/bin/anaconda3/envs/lartelt_dev/share/proj failed


Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_nbp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_nbp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_nbp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_ra+rh_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_gpp_SAT_721.pkl
Done loading data for gdf1x1_CABLE-POP_nbp_SAT_721.pkl
